```{contents}
```
## Entity Memory 

**Entity Memory** is a memory mechanism that **extracts, tracks, and updates facts about entities** (people, systems, products, organizations, issues) mentioned during a conversation.

Instead of storing raw chat history, it stores **structured knowledge about entities** and their attributes.

Provided by LangChain.

```
Conversation
   ↓
Entity Extraction
   ↓
Entity → Attributes Map
   ↓
Prompt Context (facts only)
```

---

### Why Entity Memory Exists

Traditional memories focus on *conversation flow*.
Entity memory focuses on *facts*.

It solves:

* Repetition (“Which OS again?”)
* Loss of important details in summaries
* Inefficient long-term recall

Example:

```
Entity: User
  - OS: Windows 11
  - Issue: VPN disconnects
  - Frequency: Every 5 minutes
```

---

### How Entity Memory Works Internally

1. LLM scans each turn
2. Identifies entities (noun phrases)
3. Extracts or updates facts about entities
4. Stores them in a structured store
5. Injects relevant entity facts into future prompts

```
"My VPN disconnects on Windows 11"
→ Entity: VPN (Issue: disconnects)
→ Entity: OS (Windows 11)
```

---

### Architecture View

![Image](https://miro.medium.com/1%2AOyHxTQJzuOWswO0fUOmi9A.png?utm_source=chatgpt.com)

![Image](https://pbs.twimg.com/media/FnHayTUaUAIaTg2.jpg?utm_source=chatgpt.com)

![Image](https://dist.neo4j.com/wp-content/uploads/20240722075316/knowledge-graph-example-1.png?utm_source=chatgpt.com)

---

### Entity Memory Demonstration (LangChain)

#### Initialize Entity Memory



In [1]:
from langchain_classic.memory import ConversationEntityMemory
from langchain_openai import OpenAI

llm = OpenAI()

memory = ConversationEntityMemory(
    llm=llm
)

C:\Users\sangouda\AppData\Local\Temp\ipykernel_21264\2339198863.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationEntityMemory(
c:\Users\sangouda\Python3.10\lib\site-packages\pydantic\main.py:250: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)




This memory:

* Uses the LLM to extract entities
* Maintains an entity → facts dictionary

---

#### Attach Memory to a Conversation Chain



In [6]:
from langchain_classic.chains import ConversationChain
from langchain_core.prompts import PromptTemplate

# Custom prompt template that includes entities variable
template = """You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, you are a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs help with a specific question or just wants to have a conversation about a particular topic, you are here to assist.

Context:
{entities}

Current conversation:
{history}
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables=["entities", "history", "input"],
    template=template
)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
)



---

#### Run a Conversation



In [7]:
conversation.predict(input="My VPN disconnects every 5 minutes on Windows 11")
conversation.predict(input="It started after the last update")
conversation.predict(input="What OS am I using?")


' You are using Windows 11 as your operating system. Is there anything else I can assist you with?'



**Expected Output**

```
You are using Windows 11.
```

The model answers from **entity memory**, not chat replay.

---

### Inspecting Stored Entities

```python
print(memory.entity_store.store)
```

Example output:

```python
{
  "VPN": "Disconnects every 5 minutes, started after last update",
  "Operating System": "Windows 11"
}
```

This is **structured factual memory**.

---

### Prompt Injection Mechanism

Instead of raw history, the prompt contains:

```
Known Entities:
- Operating System: Windows 11
- VPN: Disconnects every 5 minutes, started after last update

H: What OS am I using?
AI:
```

---

### Strengths of Entity Memory

* Fact-focused
* Low token usage
* Persistent knowledge
* Reduces repetitive questions
* Excellent for personalization

---

### Limitations

| Limitation          | Explanation              |
| ------------------- | ------------------------ |
| Extraction errors   | Depends on LLM quality   |
| No temporal order   | Facts overwrite history  |
| Not semantic search | Exact entity recall only |
| Schema-free         | Can become inconsistent  |

---

### When to Use Entity Memory

Use it when:

* User attributes matter
* Facts must persist
* Personalization is needed
* Conversations are long-lived

Avoid it when:

* Narrative flow matters
* Exact dialogue history is needed
* Auditing is required

---

### Comparison with Other Memory Types

| Memory Type       | Stores               | Best For            |
| ----------------- | -------------------- | ------------------- |
| Buffer            | Full chat            | Short conversations |
| Summary           | Compressed chat      | Long dialogue       |
| VectorStore       | Semantic recall      | Long-term memory    |
| **Entity Memory** | Facts about entities | Personalization     |

---

### Real-World Use Case

**IT Support Assistant**

* Remembers user OS, device, location
* Tracks recurring issues per system
* Avoids asking same diagnostic questions
* Feels "aware" across sessions

Often combined with:

* Vector memory (incident history)
* Window memory (recent context)

---

### Key Takeaways

* Entity Memory stores **facts, not dialogue**
* It extracts structured knowledge automatically
* Extremely efficient for long-term personalization
* Critical for production-grade conversational agents
